In [1]:
import threading
import asyncio
import queue
import time
import sounddevice as sd
import numpy as np
import torch
from transformers import AutoTokenizer
from parler_tts import ParlerTTSForConditionalGeneration

# === Device & Model Setup ===
DEVICE = torch.device("cuda")

TOKENIZER = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
MODEL_1 = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(DEVICE)
MODEL_2 = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(DEVICE)

DESC_TOKENIZER = AutoTokenizer.from_pretrained(MODEL_1.config.text_encoder._name_or_path)
DESCRIPTION = "Rohit's voice is monotone yet slightly fast in delivery, with minimal background noise."
DESC_INPUTS = DESC_TOKENIZER(DESCRIPTION, return_tensors="pt").to(DEVICE)
DESC_INPUT_IDS = DESC_INPUTS.input_ids
DESC_ATTN_MASK = DESC_INPUTS.attention_mask

SAMPLING_RATE = MODEL_1.config.sampling_rate
sd.default.latency = 'low'

# === TTS Worker ===
def tts_worker(text_queue, audio_dict, model, audio_event_dict):
    while True:
        item = text_queue.get()
        if item is None:
            break

        seq_id, text, start_time = item

        prompt_inputs = TOKENIZER(text, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            audio_tensor = model.generate(
                input_ids=DESC_INPUT_IDS,
                attention_mask=DESC_ATTN_MASK,
                prompt_input_ids=prompt_inputs.input_ids,
                prompt_attention_mask=prompt_inputs.attention_mask
            )

        audio = audio_tensor.cpu().numpy().squeeze()
        if audio.ndim > 1:
            audio = audio.flatten()
        audio = audio.astype(np.float32)
        audio /= np.max(np.abs(audio)) if np.max(np.abs(audio)) else 1.0

        audio_dict[seq_id] = (audio, text, start_time)
        audio_event_dict[seq_id].set()

# === Ordered Playback ===
def ordered_playback(audio_dict, audio_event_dict, total_chunks, latency_records):
    for seq_id in range(total_chunks):
        audio_event_dict[seq_id].wait()
        audio, text, start_time = audio_dict[seq_id]

        latency = time.time() - start_time
        latency_records.append((text, latency))
        print(f"\n🔊 Playing chunk {seq_id+1}: '{text[:30]}...' | Latency: {latency:.2f}s")

        sd.play(audio, samplerate=SAMPLING_RATE)
        sd.wait()

    print(" Playback complete.")

# === Dynamic Chunking ===
def dynamic_chunking(text, initial_size, max_size):
    chunks, toggle, sizes = [], 0, [initial_size, initial_size]
    i = 0

    while i < len(text):
        chunk_size = sizes[toggle % 2]
        chunk = text[i:i+chunk_size].strip()
        chunks.append(chunk)
        i += chunk_size

        # Double chunk size on each toggle until reaching max_size
        sizes[toggle % 2] = min(sizes[toggle % 2] * 2, max_size)
        toggle += 1

    return chunks

# === Main Function ===
def parallel_ordered_tts_dynamic_chunks(prompt, initial_chunk_size=50, max_chunk_size=1000):
    text_chunks = dynamic_chunking(prompt, initial_chunk_size, max_chunk_size)
    total_chunks = len(text_chunks)

    text_queue1, text_queue2 = queue.Queue(), queue.Queue()
    audio_dict = {}
    audio_event_dict = {i: threading.Event() for i in range(total_chunks)}
    latency_records = []

    start_time = time.time()
    for seq_id, chunk in enumerate(text_chunks):
        target_queue = text_queue1 if seq_id % 2 == 0 else text_queue2
        target_queue.put((seq_id, chunk, time.time()))

    text_queue1.put(None)
    text_queue2.put(None)

    # TTS Workers
    t1 = threading.Thread(target=tts_worker, args=(text_queue1, audio_dict, MODEL_1, audio_event_dict))
    t2 = threading.Thread(target=tts_worker, args=(text_queue2, audio_dict, MODEL_2, audio_event_dict))
    t1.start()
    t2.start()

    # Ordered Playback
    ordered_playback(audio_dict, audio_event_dict, total_chunks, latency_records)

    t1.join()
    t2.join()

    total_duration = time.time() - start_time

    # Latency report
    print("\n Per-Chunk Latency Report:")
    for idx, (chunk, latency) in enumerate(latency_records, 1):
        print(f"Chunk {idx}: Latency = {latency:.2f}s | '{chunk[:50]}...'")

    print(f"\n Total Streaming Time: {total_duration:.2f}s")

# === Example Usage ===
if __name__ == "__main__":
    hindi_text = (
        "भारत एक विशाल देश है जो विभिन्न संस्कृतियों, परंपराओं, भाषाओं और प्राकृतिक सौंदर्य से भरा हुआ है। "
        "यहाँ की विविधता इसकी पहचान है और यह देश को विश्व स्तर पर विशिष्ट बनाती है। "
        "यहां उत्तर में हिमालय पर्वत श्रृंखला है और दक्षिण में हिंद महासागर इसे घेरता है। "
        "भारत का इतिहास प्राचीन सभ्यताओं से लेकर आधुनिक तकनीकी युग तक फैला हुआ है। "
        "इसकी संस्कृति विभिन्न त्यौहारों, कला, संगीत, नृत्य और भोजन में झलकती है।"
    )

    parallel_ordered_tts_dynamic_chunks(
        hindi_text, 
        initial_chunk_size=10, 
        max_chunk_size=1000
    )


Flash attention 2 is not installed
ParlerTTSForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
ParlerTTSForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten.


🔊 Playing chunk 1: 'भारत एक वि...' | Latency: 1.70s

🔊 Playing chunk 2: 'शाल देश है...' | Latency: 3.42s

🔊 Playing chunk 3: 'जो विभिन्न संस्कृति...' | Latency: 4.85s

🔊 Playing chunk 4: 'यों, परंपराओं, भाषाओ...' | Latency: 6.78s

🔊 Playing chunk 5: 'ं और प्राकृतिक सौंदर्य से भरा ...' | Latency: 8.91s

🔊 Playing chunk 6: 'ाँ की विविधता इसकी पहचान है और...' | Latency: 12.46s

🔊 Playing chunk 7: 'विश्व स्तर पर विशिष्ट बनाती है...' | Latency: 16.59s

🔊 Playing chunk 8: 'िण में हिंद महासागर इसे घेरता ...' | Latency: 23.24s

🔊 Playing chunk 9: 'तकनीकी युग तक फैला हुआ है। इसक...' | Latency: 29.87s
 Playback complete.

 Per-Chunk Latency Report:
Chunk 1: Latency = 1.70s | 'भारत एक वि...'
Chunk 2: Latency = 3.42s | 'शाल देश है...'
Chunk 3: Latency = 4.85s | 'जो विभिन्न संस्कृति...'
Chunk 4: Latency = 6.78s | 'यों, परंपराओं, भाषाओ...'
Chunk 5: Latency = 8.91s | 'ं और प्राकृतिक सौंदर्य से भरा हुआ है। यह...'
Chunk 6: Latency = 12.46s | 'ाँ की विविधता इसकी पहचान है और यह देश को...'
Chunk 7: Laten

In [6]:
import threading
import queue
import time
import numpy as np
import torch
from transformers import AutoTokenizer
from parler_tts import ParlerTTSForConditionalGeneration
from scipy.io.wavfile import write as wav_write

# === Device & Model Setup ===
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TOKENIZER = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
MODEL_1 = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(DEVICE)
MODEL_2 = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(DEVICE)

DESC_TOKENIZER = AutoTokenizer.from_pretrained(MODEL_1.config.text_encoder._name_or_path)
DESCRIPTION = "Rohit's voice is monotone yet slightly fast in delivery, with minimal background noise."
DESC_INPUTS = DESC_TOKENIZER(DESCRIPTION, return_tensors="pt").to(DEVICE)
DESC_INPUT_IDS = DESC_INPUTS.input_ids
DESC_ATTN_MASK = DESC_INPUTS.attention_mask

SAMPLING_RATE = MODEL_1.config.sampling_rate

# === TTS Worker for WAV Generation ===
def tts_worker(text_queue, audio_dict, model, audio_event_dict):
    while True:
        item = text_queue.get()
        if item is None:
            break

        seq_id, text = item

        prompt_inputs = TOKENIZER(text, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            audio_tensor = model.generate(
                input_ids=DESC_INPUT_IDS,
                attention_mask=DESC_ATTN_MASK,
                prompt_input_ids=prompt_inputs.input_ids,
                prompt_attention_mask=prompt_inputs.attention_mask
            )

        audio = audio_tensor.cpu().numpy().squeeze()
        if audio.ndim > 1:
            audio = audio.flatten()
        audio = audio.astype(np.float32)
        audio /= np.max(np.abs(audio)) if np.max(np.abs(audio)) else 1.0

        audio_dict[seq_id] = audio
        audio_event_dict[seq_id].set()

# === Dynamic Chunking ===
def dynamic_chunking(text, initial_size, max_size):
    chunks, toggle, sizes = [], 0, [initial_size, initial_size]
    i = 0

    while i < len(text):
        chunk_size = sizes[toggle % 2]
        chunk = text[i:i+chunk_size].strip()
        chunks.append(chunk)
        i += chunk_size

        # Double chunk size on each toggle until reaching max_size
        sizes[toggle % 2] = min(sizes[toggle % 2] * 2, max_size)
        toggle += 1

    return chunks

# === Main Function for Audio Generation ===
def generate_wav_from_text(prompt, initial_chunk_size=50, max_chunk_size=1000, output_file="generated_audio.wav"):
    text_chunks = dynamic_chunking(prompt, initial_chunk_size, max_chunk_size)
    total_chunks = len(text_chunks)

    text_queue1, text_queue2 = queue.Queue(), queue.Queue()
    audio_dict = {}
    audio_event_dict = {i: threading.Event() for i in range(total_chunks)}

    # Distribute text chunks
    for seq_id, chunk in enumerate(text_chunks):
        target_queue = text_queue1 if seq_id % 2 == 0 else text_queue2
        target_queue.put((seq_id, chunk))

    text_queue1.put(None)
    text_queue2.put(None)

    # TTS worker threads
    t1 = threading.Thread(target=tts_worker, args=(text_queue1, audio_dict, MODEL_1, audio_event_dict))
    t2 = threading.Thread(target=tts_worker, args=(text_queue2, audio_dict, MODEL_2, audio_event_dict))
    t1.start()
    t2.start()

    # Wait for TTS generation
    t1.join()
    t2.join()

    # Ensure audio is in correct order
    audio_chunks = []
    for seq_id in range(total_chunks):
        audio_event_dict[seq_id].wait()
        audio_chunks.append(audio_dict[seq_id])

    # Concatenate audio chunks
    combined_audio = np.concatenate(audio_chunks)

    # Normalize audio
    combined_audio /= np.max(np.abs(combined_audio)) if np.max(np.abs(combined_audio)) else 1.0

    # Save audio as WAV file
    wav_write(output_file, SAMPLING_RATE, (combined_audio * 32767).astype(np.int16))

    print(f"Audio successfully saved as '{output_file}'")



Flash attention 2 is not installed
ParlerTTSForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
ParlerTTSForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten.

KeyboardInterrupt: 

In [6]:
hindi_text = ("""आधुनिक सूचना तकनीक के युग में कृत्रिम बुद्धिमत्ता यानी आर्टिफिशियल इंटेलिजेंस ने मानव जीवन में क्रांतिकारी परिवर्तन ला दिया है।
मशीन लर्निंग, जो कृत्रिम बुद्धिमत्ता की एक उप-शाखा है, डाटा के माध्यम से स्वतः ही सीखने और भविष्यवाणी करने की क्षमता रखती है।
न्यूरल नेटवर्क और डीप लर्निंग तकनीकें मानव मस्तिष्क की संरचना से प्रेरित हैं, जिनमें कृत्रिम न्यूरॉन आपस में जटिल गणितीय प्रक्रियाओं के माध्यम से संवाद करते हुए निर्णय लेते हैं।"""
     
)

initial_chunk_size = 50  # Initial chunk size
max_chunk_size = 1000    # Max chunk size

generate_wav_from_text(
    prompt=hindi_text,
    initial_chunk_size=initial_chunk_size,
    max_chunk_size=max_chunk_size,
    output_file=f"generated_audio_2_{initial_chunk_size}.wav"
)


✅ Audio successfully saved as 'generated_audio_2_50.wav'


In [7]:
for i in range(10,60,10):
    hindi_text = ("""इन तकनीकों का उपयोग कंप्यूटर विज़न, प्राकृतिक भाषा संसाधन और स्वचालित वाहन प्रणाली जैसे क्षेत्रों में व्यापक रूप से किया जा रहा है। हाल ही में विकसित किए गए चैटबॉट्स और जनरेटिव प्री-ट्रेन्ड ट्रांसफॉर्मर मॉडल, जिन्हें GPT कहा जाता है, मानव भाषाओं को समझने तथा उनके संदर्भ के अनुरूप अर्थपूर्ण जवाब उत्पन्न करने में सक्षम हैं। यद्यपि यह तकनीकें अत्यंत लाभकारी हैं, फिर भी डेटा की गोपनीयता, नैतिकता, तथा रोजगार पर इनका प्रभाव जैसे जटिल प्रश्न भी उत्पन्न होते हैं, जिनका समाधान आने वाले समय में मानव और मशीन के मध्य संबंधों की दिशा निर्धारित करेगा।""")

    initial_chunk_size = i  # Initial chunk size
    max_chunk_size = 1000    # Max chunk size

    generate_wav_from_text(
    prompt=hindi_text,
    initial_chunk_size=initial_chunk_size,
    max_chunk_size=max_chunk_size,
    output_file=f"generated_audio_3_{initial_chunk_size}.wav")

    

✅ Audio successfully saved as 'generated_audio_3_10.wav'
✅ Audio successfully saved as 'generated_audio_3_20.wav'
✅ Audio successfully saved as 'generated_audio_3_30.wav'
✅ Audio successfully saved as 'generated_audio_3_40.wav'
✅ Audio successfully saved as 'generated_audio_3_50.wav'


In [9]:
for i in range(10,60,10):
    hindi_text = ("""मानव शरीर एक अत्यंत जटिल तंत्र है जिसमें विभिन्न अंग आपस में सामंजस्य के साथ कार्य करते हैं। हमारा हृदय रक्त परिसंचरण प्रणाली का केंद्र है, जो पूरे शरीर में ऑक्सीजन और पोषक तत्वों की आपूर्ति सुनिश्चित करता है। रक्त वाहिकाओं का जाल धमनियों और शिराओं के माध्यम से रक्त को हर अंग तक पहुँचाता है। इसके अलावा, तंत्रिका तंत्र शरीर के हर हिस्से को नियंत्रित करता है, जिसमें मस्तिष्क, रीढ़ की हड्डी और परिधीय तंत्रिकाएँ प्रमुख भूमिका निभाती हैं। """)

    initial_chunk_size = i  # Initial chunk size
    max_chunk_size = 1000    # Max chunk size

    generate_wav_from_text(
    prompt=hindi_text,
    initial_chunk_size=initial_chunk_size,
    max_chunk_size=max_chunk_size,
    output_file=f"generated_audio_4_{initial_chunk_size}.wav")

    

✅ Audio successfully saved as 'generated_audio_4_10.wav'
✅ Audio successfully saved as 'generated_audio_4_20.wav'
✅ Audio successfully saved as 'generated_audio_4_30.wav'
✅ Audio successfully saved as 'generated_audio_4_40.wav'
✅ Audio successfully saved as 'generated_audio_4_50.wav'


In [10]:
for i in range(10,60,10):
    hindi_text = ('''एक समय की बात है, दूर किसी गाँव में एक बूढ़ा किसान अपने बेटे के साथ रहता था। उनके पास एक घोड़ा था, जो गाँव भर में अपनी खूबसूरती और ताकत के लिए प्रसिद्ध था। एक दिन वह घोड़ा अचानक लापता हो गया। गाँव के लोग यह खबर सुनकर किसान के पास आए और बोले, "तुम्हारे साथ बड़ी दुर्भाग्यपूर्ण घटना घटी है।" किसान ने सहजता से जवाब दिया, "शायद ये अच्छा है या बुरा, कौन जाने?" कुछ दिनों बाद, किसान का घोड़ा वापस आया, और अपने साथ जंगली घोड़ों का एक पूरा झुंड ले आया। गाँव के लोग फिर किसान के पास पहुँचे और बोले, "तुम्हारा भाग्य बहुत अच्छा है। एक घोड़े के बदले अब तुम्हारे पास कई घोड़े आ गए हैं।" किसान फिर मुस्कुराते हुए बोला, "शायद अच्छा है, शायद नहीं, कौन जाने?''')

    initial_chunk_size = i  # Initial chunk size
    max_chunk_size = 1000    # Max chunk size

    generate_wav_from_text(
    prompt=hindi_text,
    initial_chunk_size=initial_chunk_size,
    max_chunk_size=max_chunk_size,
    output_file=f"generated_audio_5_{initial_chunk_size}.wav")

    

✅ Audio successfully saved as 'generated_audio_5_10.wav'
✅ Audio successfully saved as 'generated_audio_5_20.wav'
✅ Audio successfully saved as 'generated_audio_5_30.wav'
✅ Audio successfully saved as 'generated_audio_5_40.wav'
✅ Audio successfully saved as 'generated_audio_5_50.wav'


In [3]:
for i in range(40,60,10):
    hindi_text = ('''कुछ दिन बीते, और किसान का बेटा उन नए जंगली घोड़ों को प्रशिक्षित करते समय गिर पड़ा और उसकी टाँग टूट गई। फिर से लोग सहानुभूति जताने आए और बोले, "ये तो बहुत बुरा हुआ, अब तुम्हारा बेटा खेती-बाड़ी में मदद नहीं कर पाएगा।" किसान ने उसी शांति के साथ उत्तर दिया, "अच्छा है या बुरा, ये तो समय बताएगा।" अचानक गाँव में युद्ध की घोषणा हो गई। राजा के सिपाही गाँव-गाँव जाकर युवकों को सेना में भर्ती करने लगे। किसान के बेटे की टूटी टाँग देखकर उन्होंने उसे भर्ती नहीं किया। गाँव के युवक सेना में गए, जिनमें से कई युद्ध से वापस नहीं लौटे। गाँव के लोग फिर किसान के पास आए और बोले, "तुम्हारी दूरदर्शिता सच साबित हुई। तुम्हारा बेटा सुरक्षित है।" किसान ने फिर उसी शांत मुस्कान के साथ कहा, "अच्छा हुआ या बुरा, इसे सिर्फ़ समय ही बता सकता है।" यह कहानी हमें सिखाती है कि जीवन की हर परिस्थिति में तुरंत फैसला न करें। क्योंकि जो घटना आज हमें बुरी लगती है, शायद वही कल हमारे लिए अच्छी साबित हो जाए।''')

    initial_chunk_size = i  # Initial chunk size
    max_chunk_size = 1000    # Max chunk size

    generate_wav_from_text(
    prompt=hindi_text,
    initial_chunk_size=initial_chunk_size,
    max_chunk_size=max_chunk_size,
    output_file=f"generated_audio_6_{initial_chunk_size}.wav")

`prompt_attention_mask` is specified but `attention_mask` is not. A full `attention_mask` will be created. Make sure this is the intended behaviour.
`prompt_attention_mask` is specified but `attention_mask` is not. A full `attention_mask` will be created. Make sure this is the intended behaviour.


✅ Audio successfully saved as 'generated_audio_6_40.wav'
✅ Audio successfully saved as 'generated_audio_6_50.wav'


In [4]:
for i in range(10,60,10):
    hindi_text = ('''मुझे पानी पीना है। क्या आप मेरी मदद कर सकते हैं? आज मौसम बहुत अच्छा है। यह किताब मुझे बहुत पसंद है।''')

    initial_chunk_size = i  # Initial chunk size
    max_chunk_size = 1000    # Max chunk size

    generate_wav_from_text(
    prompt=hindi_text,
    initial_chunk_size=initial_chunk_size,
    max_chunk_size=max_chunk_size,
    output_file=f"generated_audio_7_{initial_chunk_size}.wav")

✅ Audio successfully saved as 'generated_audio_7_10.wav'
✅ Audio successfully saved as 'generated_audio_7_20.wav'
✅ Audio successfully saved as 'generated_audio_7_30.wav'
✅ Audio successfully saved as 'generated_audio_7_40.wav'
✅ Audio successfully saved as 'generated_audio_7_50.wav'


In [5]:

for i in range(10,60,10):
    hindi_text = ('''चलो पार्क में चलते हैं। मुझे थोड़ी देर सोना है। तुम कहाँ जा रहे हो?''')

    initial_chunk_size = i  # Initial chunk size
    max_chunk_size = 1000    # Max chunk size

    generate_wav_from_text(
    prompt=hindi_text,
    initial_chunk_size=initial_chunk_size,
    max_chunk_size=max_chunk_size,
    output_file=f"generated_audio_8_{initial_chunk_size}.wav")

✅ Audio successfully saved as 'generated_audio_8_10.wav'
✅ Audio successfully saved as 'generated_audio_8_20.wav'
✅ Audio successfully saved as 'generated_audio_8_30.wav'
✅ Audio successfully saved as 'generated_audio_8_40.wav'
✅ Audio successfully saved as 'generated_audio_8_50.wav'


In [6]:

for i in range(10,60,10):
    hindi_text = ('''वह बहुत समझदार बच्चा है। यह काम अब पूरा हो गया है। धन्यवाद, आपने समय निकाला।''')

    initial_chunk_size = i  # Initial chunk size
    max_chunk_size = 1000    # Max chunk size

    generate_wav_from_text(
    prompt=hindi_text,
    initial_chunk_size=initial_chunk_size,
    max_chunk_size=max_chunk_size,
    output_file=f"generated_audio_9_{initial_chunk_size}.wav")

✅ Audio successfully saved as 'generated_audio_9_10.wav'
✅ Audio successfully saved as 'generated_audio_9_20.wav'
✅ Audio successfully saved as 'generated_audio_9_30.wav'
✅ Audio successfully saved as 'generated_audio_9_40.wav'
✅ Audio successfully saved as 'generated_audio_9_50.wav'


In [8]:
for i in range(10,60,10):
    hindi_text = ('''यह तो बहुत अच्छी खबर है! मुझे आज बहुत खुशी हो रही है। मुझे अकेलापन महसूस हो रहा है। आज का दिन कुछ अच्छा नहीं गया। तुमने मेरी बात क्यों नहीं मानी? यह बिल्कुल भी ठीक नहीं हुआ।''')

    initial_chunk_size = i  # Initial chunk size
    max_chunk_size = 1000    # Max chunk size

    generate_wav_from_text(
    prompt=hindi_text,
    initial_chunk_size=initial_chunk_size,
    max_chunk_size=max_chunk_size,
    output_file=f"generated_audio_10_{initial_chunk_size}.wav")

✅ Audio successfully saved as 'generated_audio_10_10.wav'
✅ Audio successfully saved as 'generated_audio_10_20.wav'
✅ Audio successfully saved as 'generated_audio_10_30.wav'
✅ Audio successfully saved as 'generated_audio_10_40.wav'
✅ Audio successfully saved as 'generated_audio_10_50.wav'


In [5]:
!python ./UTMOSv2/inference.py --input_dir /media/irlab/ba5a20df-2f59-4a88-b4ad-08a3df254e15/Parth/Voice/gen_audios --out_path /media/irlab/ba5a20df-2f59-4a88-b4ad-08a3df254e15/Parth/Voice/gen_audios/result.csv


                                           file_path  ... predict_dataset
0  /media/irlab/ba5a20df-2f59-4a88-b4ad-08a3df254...  ...         sarulab
1  /media/irlab/ba5a20df-2f59-4a88-b4ad-08a3df254...  ...         sarulab
2  /media/irlab/ba5a20df-2f59-4a88-b4ad-08a3df254...  ...         sarulab
3  /media/irlab/ba5a20df-2f59-4a88-b4ad-08a3df254...  ...         sarulab
4  /media/irlab/ba5a20df-2f59-4a88-b4ad-08a3df254...  ...         sarulab

[5 rows x 4 columns]
Using model: ssl_multispec_ext_v2
/home/irlab/.pyenv/versions/pyvoice/lib/python3.10/site-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Loaded weight from /home/irlab/.cache/utmosv2/models/fusion_stage3/fold0_s42_best_model.pth
